In [18]:
petersburg = [0.0974, 0.1352, 0.0817, 0.1016, 0.0968, 0.1064, 0.105]
magadan = [0.1033, 0.0915, 0.0781, 0.0685, 0.0677, 0.0697, 0.0764, 0.0689]

In [20]:
# Объединяем два списка в DataFrame
import pandas as pd
import numpy as np

df = pd.DataFrame(petersburg, columns=['Санкт-Петербург'])
df = pd.concat([df, pd.Series(magadan, name='Магадан')], axis=1)

# Заполняем пропущенные значения медианным значением
df = df.fillna(df['Санкт-Петербург'].median())
print(df)

   Санкт-Петербург  Магадан
0           0.0974   0.1033
1           0.1352   0.0915
2           0.0817   0.0781
3           0.1016   0.0685
4           0.0968   0.0677
5           0.1064   0.0697
6           0.1050   0.0764
7           0.1016   0.0689


Альтернативная гипотеза: В зависимости от города-производителя размер раковин мидий изменятеся
Нулевая гипотеза: Вне зависимости от города-производителя размер раковин мидий не изменяется

Проверяем данные на нормальность распределения, для выбора теста на корреляцию.

In [21]:
H0 = 'Данные распределены нормально'
Ha = 'Данные не распределены нормально (мы отвергаем H0)'
alpha = 0.05

In [22]:
# Проводим тест Шапиро-Уилка

from scipy.stats import shapiro

_, p = shapiro(df)
print('p=%.3f' % p)

# Интерпретация 

if p > alpha:
	print(H0)
else:
	print(Ha)

p=0.101
Данные распределены нормально


In [23]:
# Проводим тест Д’Агостино

from scipy.stats import normaltest

_, p = normaltest(df)
print('p=%.3f' % p[0])

# Интерпретация 

if p[0] > alpha/2:
	print(H0)
else:
	print(Ha)

p=0.035
Данные распределены нормально


/home/aleksandr/anaconda3/lib/python3.8/site-packages/scipy/stats/stats.py:1603: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=8
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


Так как тип зависимой переменной - числовая (количественная) и распределение нормальное, то подойдет параметрический тест на корреляцию Пирсона

In [24]:
# Вычислим коэффициент корреляции Пирсона между переменными в датасете

round(df.corr(), 2)

,Санкт-Петербург,Магадан
Санкт-Петербург,1.00,0.26
Магадан,0.26,1.00


Коэффициент корреляции = 0.26. Корреляция очень слабая. Зависимость между размерами раковин в разных городах очень слабая.

Так как тип зависимой переменной (размер раковины) - числовой (количественный). Группы происходят из разных совокупностей (города). Следовательно, мы используем независимый T-тест для сравнения выборок

In [25]:
from scipy.stats import ttest_ind

H0 = 'Нет значимой разницы между размерами раковин в разных городах.'
Ha = 'Есть значимая разница между размерами раковин в разных городах.'

def t_test(df):
    print('\n' + "*** Результаты независимого T-теста ***")
    test_results = ttest_ind(df['Санкт-Петербург'], df['Магадан'])

    p = round(test_results[1],2)

    if p>alpha:
        print(f"{p} > {alpha}. Мы не можем отвергнуть нулевую гипотезу. {H0}")
    else:
        print(f"{p} <= {alpha}. Мы отвергаем нулевую гипотезу. {Ha}")

t_test(df)


*** Результаты независимого T-теста ***
0.0 <= 0.05. Мы отвергаем нулевую гипотезу. Есть значимая разница между размерами раковин в разных городах.


In [27]:
# разница в среднем размере мидии в зависимости от города-производителя
import statistics 

round(statistics.mean(petersburg) - statistics.mean(magadan), 4)

0.0254

Средний размер раковин мидий в Санкт-Петербурге больше чем в Магадане на 0.0254